## Building a Text-Based Data Set by Web Scraping
This notebook creates a text data set stored in a database. (WAS UNSUCCESSFUL)

### Imports

In [1]:
import sqlite3
import requests #requests package to get the pages

from bs4 import BeautifulSoup #beautiful soup to process/parse the pages
from bs4.element import Comment

import datetime

### Builds a database with the following fields

* `dt`: the date and time when we pulled the page.
* `url`: the specific URL we pulled from contained in anasoundcloud_websites.txt. Ex. https://soundcloud.com/anewangle/sea-change-8-nichole-heyer.
* `text`: the text of the `url`. 
* `pulled`: A boolean that is TRUE if we've tried to pull the text from the URL. This is useful for keeping track.

In [2]:
db = sqlite3.connect("anasoundcloud_database.db") #creates database
cur = db.cursor() #cursor allows us to talk to the database

In [3]:
#creates table in database
cur.execute('''DROP TABLE IF EXISTS site_text''')
cur.execute('''CREATE TABLE site_text ( 
    dt DATETIME,
    url TEXT,
    text TEXT,
    pulled BOOLEAN)''')
db.commit()

In [ ]:
input_file = "anasoundcloud_dataset_clean.txt"

#fills table with data
with open(input_file,'r', encoding="Latin-1") as ifile:
    for idx, line in enumerate(ifile.readlines()):
        line = line.strip().split(",")
        cur.execute('''
            INSERT INTO site_text (url, text, pulled)
            VALUES(?,?,?)''', line)
        
db.commit()

### Reads in list of websites from anasoundcloud_websites.txt file
Tested my code using the "anasoundcould_test.txt" file which only contains 1 url before running it on all of the urls. "anasoundcould_websites.txt" contains all urls for all episodes through November 26, 2019.

In [4]:
#reads in list of A New Angle Soundcloud links as `sites`
sites = []

with open("anasoundcloud_test.txt",'r') as infile :
#with open("anasoundcloud_websites.txt",'r') as infile :
    for line in infile :
        sites.append(line.strip())

In [ ]:
#prints all links in `sites`
print(sites)
r = requests.get(sites[0])
#checks HTTP response status codes -> 200 = good
#r.status_code

### Extracts visible text

In [ ]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]', 'div']:
        return False
    if isinstance(element, Comment):
        return False
    return True

In [ ]:
page_text = ''

for url in sites :
    try :
        r = requests.get(site)
        if r.status_code == 200 :
            soup = BeautifulSoup(r.text, 'html.parser')
            texts = soup.findAll(text=True)
            visible_texts = filter(tag_visible, texts) 
            page_text = " ".join(t.strip() for t in visible_texts)

        sql_query = ('INSERT INTO site_text(base_url, url, text, pulled)'
                     'VALUES(?,?,?,?)')
        cur.execute(sql_query,[sites[0],site, page_text,'1'])

        print("Completed pull for {} .".format(site))

        db.commit()
    except :
        pass ;

In [ ]:
page_text

In [ ]:
for url in sites :
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    links = [url] # make sure the main page is in there. 
    
    # Now we have all the links. Let's just load them into the DB. We'll do the 
    # page pulls and parsing separately. 
    
    for link in set(links) : # wrap a `set` around it so that we don't get duplicates
        new_row = [datetime.datetime.now(),
                   url,
                   link,
                   "", # empty string for text
                   False]
        
        cur.execute('''INSERT INTO site_text (dt,url,text,pulled) 
               VALUES (?,?,?,?)''',new_row)
        
db.commit()

In [5]:
#extract text and stores it in a dictionary that has the url as the key and the value is the text.
#from John via stackoverflow
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]', 'div']:
        return False
    if isinstance(element, Comment):
        return False
    return True

In [6]:
anasc_text = ''

for link in sites :
    try :
        r = requests.get(link)
    except :
        pass 
    
    if r.status_code == 200 :
        soup = BeautifulSoup(r.text, 'html.parser')
        texts = soup.findAll(text=True)
        visible_texts = filter(tag_visible, texts) 
        anasc_text[link] = " ".join(t.strip() for t in visible_texts)

TypeError: 'str' object does not support item assignment